In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import astropy.constants as c
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from scipy.signal import peak_widths
from scipy.optimize import curve_fit
import math
from statistics import stdev
from scipy.integrate import quad
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

In [2]:
data=np.genfromtxt('Kypointsrange(0, 20).txt', delimiter=' ')
extradata=np.genfromtxt('Kotherstuffrange(0, 20).txt', delimiter=' ')

Stars = np.linspace(1, 50, 50)
GG=np.genfromtxt('Star Quiz data 50 stars 6 testers',delimiter=' ')
GG=np.array(GG)
Quizdata=GG.reshape((int(6),len(Stars)))
print(Quizdata.shape)

(6, 50)


In [3]:
def most_frequent(List):
    List=list(List)
    return max(set(List), key = List.count)

In [4]:
mode = most_frequent(Quizdata[0])
print(mode)

1.0


In [5]:
maybe_stars=[]
yes_stars=[]

In [6]:
for n in Stars: 
    Q=most_frequent(Quizdata[:,int(n)-1])
    if Q==0:
        maybe_stars.append(n)
    if Q==2:
        yes_stars.append(n)
        
print(maybe_stars)
print(yes_stars)
        

[2.0, 5.0, 6.0, 7.0, 11.0, 14.0, 16.0, 26.0, 29.0, 30.0, 34.0, 44.0]
[1.0, 4.0, 8.0, 13.0, 15.0]


In [7]:
def bin_it(yvalues, xvalues, binsize):
    '''puts an array into averaged bins of a given size'''
    N=int(math.ceil(len(yvalues)/binsize))
    C=0
    newy=[]
    while C<N:
        if (C+1)*binsize<len(yvalues):
            Q=(np.sum(yvalues[C*binsize:(C+1)*binsize]))/binsize
        else:
            Q=(np.sum(yvalues[C*binsize:len(yvalues)]))/binsize
        newy.append(Q)
        C+=1
    y=np.array(newy)
    x=xvalues[0::binsize] 
    return y[0:len(y)-1], x[0:len(y)-1]

In [8]:
#Adds random noise
def addnoise(pure):
    vary = np.random.uniform(0, 1, pure.shape)
    signal = -(pure* np.log(vary))
    return signal

In [9]:
def gaus(x, height, centre, width):
    '''plots a gaussian acounting for granulation'''
    return height*np.exp((-(x-centre)**2)/(2*width**2))

In [10]:
def findgran(x, y):
    #guess_vals=[np.max(y),y[round(len(y)/4)]]
    guess_vals=[0, 0]
    best_vals, covar =curve_fit(graneq, x, y, p0=guess_vals)
    return best_vals

In [11]:
def graneq(x,A,tau,SN):
    '''equation to describe the background'''
    return A/(1+(2*np.pi*x*tau)**4)+SN

In [12]:
def find_nearest(array, value):
    '''finds value closest to a given value in an array'''
    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx

In [13]:
def f(x,a,b):
    return a*x+b
def f1(x,a,b,c):
    return a*np.exp(b*x)+c

In [115]:
binsize=500
frequency = data[0]

In [121]:
binsize=250

n = 1
SN = 18*10**(-0.4*(11.0-extradata[int(n)-1,2]))

#y1,x1=data[int(n)],frequency
#y2,x2=addnoise(data[int(n)]),frequency

y,x=bin_it(addnoise(data[int(n)]),frequency,binsize)

#y=[]
#y1=[]

#for i in range(50):
#    y.append(addnoise(data[int(n)]))


    
#print(y1)

#for i in y:
#    for p in y:
#        np.sum(y[i][p])
        
        
#print(y)
#y[0][0] this gives the first value in the first array within the array of arrays
#print(y[0][:])
#print(len(y))
#print(len(y[1]))
#y=np.array(y)
#plot_y,plot_x=bin_it(avgy,frequency,binsize)
#print(y.shape)
#meanedy=np.mean(y, axis=0)
#print(meanedy.shape)
#loop over arrays
#plt.plot(frequency,addnoise(data[int(n)]))
#plt.plot(frequency,meanedy)


In [122]:
bv,covar = curve_fit(graneq,x,y,p0=[0,0,SN], bounds=([0,0,SN-SN/100],[np.inf,np.inf,SN+SN/100]))

background=graneq(x,bv[0],bv[1],bv[2])


#plt.figure()
#plt.plot(avgx,avgy)
#plt.show()

plt.figure()
plt.plot(x, y, label=f'Binned Data, Star {n}')
plt.plot(x, background, label='Background Fit')
plt.legend()
plt.show()

#plt.figure()
#plt.plot(x1, y1)
#plt.show()

#plt.figure()
#plt.plot(x2,y2)
#plt.show()

#plt.figure()
#plt.plot(x3,y3)
#plt.show()

In [123]:
%matplotlib qt

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
f = np.arange(0.0, 20000, 1) #x-axis
a0 = 2  #starting conditons for amplitude, centre and width
v0= 3000
sig0 = 50
delta_f = 5.0
s = a0*(np.exp(-0.5*((f-v0)**2/sig0**2))) #gaussian equation
l, = plt.plot(f, s, 'g', label='Gaussian Fit')
plt.plot(x, y, label=f'Binned Data, Star {n}')
plt.plot(x, background, label='Background Fit')
plt.xlabel('Frequency ($\mu Hz$)')
plt.ylabel('Power ($ppm^2 \mu Hz^{-1}$)')
plt.legend()

ax.margins(x=0)
axcolor = 'lavender'
axcent = plt.axes([0.25, 0.02, 0.65, 0.03], facecolor=axcolor) #positions of the sliders
axamp = plt.axes([0.25, 0.07, 0.65, 0.03], facecolor=axcolor)
axsig = plt.axes([0.25, 0.12, 0.65, 0.03], facecolor=axcolor)

scentre = Slider(axcent, 'nu{max}', 0.1, 17500, valinit=v0, valstep=delta_f) #parameters for sliders
scentre.valtext.set_visible(False) #removing the values so that fitting isn't as biased
samp = Slider(axamp, 'Amp', 0.1, 2, valinit=a0)
samp.valtext.set_visible(False)
ssig = Slider(axsig, 'Sigma', 0.1, 1000, valinit=sig0, valstep= delta_f)
ssig.valtext.set_visible(False)

def update(val): #function to update the amp, cent and width values as the slider is moved
    amp = samp.val
    cent = scentre.val
    sig = ssig.val
    l.set_ydata(amp*(np.exp(-0.5*((f-cent)**2/(sig**2)))))
    fig.canvas.draw_idle()
scentre.on_changed(update)
samp.on_changed(update)
ssig.on_changed(update)

integrateax = plt.axes([0.025, 0.25, 0.1, 0.04]) #position of integrate button
button1 = Button(integrateax, 'Integrate', color=axcolor, hovercolor='0.975') #creation of integrate button

def integrand_b(x, a, b): #integrate under background curve
    I = a/(1+(2*np.pi*x*b)**4)
    return I
a = bv[0] #values for A and tau
b = bv[1]
Int_background = quad(integrand_b, 0, 6000, args=(a,b))

def integrand_g(x, a, b, c):
    I = a*np.exp((-(x-b)**2)/(2*c**2))
    return I 

gauss = []
gausserr = []
background = []
backgrounderr = []
numax = []
width = []
upper = []
lower = []
def integrate(event): #defining what happens when 'integrate' button is clicked
    a = samp.val #values for a, b and c in integrand_g
    b = scentre.val
    c = ssig.val
    
    fwhm = 0.66*(scentre.val)**0.88
    upper_lim = scentre.val+fwhm/2
    lower_lim = scentre.val-fwhm/2
    
    Int_gaus = quad(integrand_g, lower_lim, upper_lim, args=(a,b,c))
    print(f'Power under Gaussian {Int_gaus}')
    print(f'Power under background {Int_background}')
    print(f'Numax is {scentre.val}')
    print(f'FWHM is {fwhm}')
    print(f'Gaussian Integral Limits are {upper_lim} and {lower_lim}')
    gauss.append(Int_gaus[0])
    gausserr.append(Int_gaus[1])
    background.append(Int_background[0])
    backgrounderr.append(Int_background[1])
    numax.append(scentre.val)
    width.append(fwhm)
    upper.append(upper_lim)
    lower.append(lower_lim)
    
button1.on_clicked(integrate)

resetax = plt.axes([0.025, 0.3, 0.1, 0.04]) #position of reset button
button2 = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975') #creating reset button
def reset(event): #defining what happens when 'integrate' button is clicked
    scentre.reset()
    samp.reset()
    ssig.reset()
button2.on_clicked(reset)



plt.show()



Power under Gaussian (971.6443525170406, 1.0787419319114206e-11)
Power under background (3534.2511872133987, 1.5144432929936613e-10)
Numax is 4880.1
FWHM is 1162.4150129229395
Gaussian Integral Limits are 5461.30750646147 and 4298.89249353853
Power under Gaussian (979.6696672120498, 1.0876518210656825e-11)
Power under background (3534.2511872133987, 1.5144432929936613e-10)
Numax is 4840.1
FWHM is 1154.0264157616634
Gaussian Integral Limits are 5417.113207880832 and 4263.086792119168
Power under Gaussian (961.5407674194287, 1.0675246991047924e-11)
Power under background (3534.2511872133987, 1.5144432929936613e-10)
Numax is 4840.1
FWHM is 1154.0264157616634
Gaussian Integral Limits are 5417.113207880832 and 4263.086792119168
Power under Gaussian (1002.9674022630783, 1.1135175029409508e-11)
Power under background (3534.2511872133987, 1.5144432929936613e-10)
Numax is 4795.1
FWHM is 1144.5792911248154
Gaussian Integral Limits are 5367.389645562408 and 4222.810354437593
Power under Gaussian 

In [128]:
average_gauss = np.mean(gauss)
average_gausserr = np.sqrt(np.sum(np.square(gausserr)))/len(gausserr)

average_background = np.array(Int_background)[0]
average_backgrounderr = np.array(Int_background)[1]

power_ratio = average_gauss/average_background
power_ratioerr = power_ratio*np.sqrt((average_gausserr/average_gauss)**2 + (average_backgrounderr/average_background)**2)

average_numax=np.mean(numax)
error_numax=np.std(numax)/np.sqrt(len(numax))
print(f'The average Numax is: {average_numax} +/- {error_numax}')

average_fwhm=np.mean(width)
error_fwhm=np.std(width)/np.sqrt(len(width))
print(f'The average FWHM is: {average_fwhm} +/- {error_fwhm}')

average_upper=np.mean(upper)
average_lower=np.mean(lower)
error_upper=np.std(upper)/np.sqrt(len(upper))
error_lower=np.std(lower)/np.sqrt(len(lower))
print(f'The average limits are, upper: {average_upper} +/- {error_upper} and lower: {average_lower} +/- {error_lower}')

print(f'Ratio of Power: {power_ratio} +/- {power_ratioerr}')


The average Numax is: 4830.1 +/- 14.352700094407323
The average FWHM is: 1151.9252853391795 +/- 3.0119172955511946
The average limits are, upper: 5406.062642669591 +/- 15.858658668202 and lower: 4254.13735733041 +/- 12.846741537957477
Ratio of Power: 0.27696122399797374 +/- 1.1947336790546897e-14
